In [30]:
%matplotlib qt
import LMP_Query
import cPickle as pickle
from sklearn import preprocessing
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM, SimpleRNN
import calendar
import holidays
import matplotlib.pyplot as plt
def work_day_or_holiday(date):
    us_holidays = holidays.UnitedStates()
    if date in us_holidays or weekday_of_date(date) == "Sunday" or weekday_of_date(date) == "Saturday":
        return int(1)
    else: return int(0)
    
def weekday_of_date(date):
    return calendar.day_name[date.weekday()]
    
def encode_onehot(df, cols):
    enc = preprocessing.OneHotEncoder()
    index = df[cols].index
    data = enc.fit_transform(df[cols].reshape(-1, 1)).toarray()
    one_hot_df = pd.DataFrame(data=data, index=index, columns=[cols + '%s' % i for i in range(data.shape[1])])
    del df[cols]
    return df.join(one_hot_df, how='inner')

In [2]:
# Retrieve sample data frame created in Spark_Time_series_preprocessing
f = open('crr_sample_nodes.pkl', 'r')
df = pickle.load(f)
f.close()

In [23]:
# Construct time series features and standardize time series
standard_scaler = preprocessing.StandardScaler()
scaled_df = pd.DataFrame(standard_scaler.fit_transform(df), index=df.index, columns=df.columns)
time_features =[]
for dt in scaled_df.index:
    time_features.append([work_day_or_holiday(dt), dt.hour, dt.weekday(), dt.month])
time_features = pd.DataFrame(time_features, index = df.index, columns=['holiday', 'hour', 'weekday', 'month'])
features_df = pd.concat([scaled_df, time_features], axis=1)
features_df = encode_onehot(features_df, 'weekday')
features_df = encode_onehot(features_df, 'month')
features_df = encode_onehot(features_df, 'hour')

In [24]:
re_scaled_df = standard_scaler.inverse_transform(scaled_df)
re_scaled_df

array([[ 27.58,   0.  ,  26.2 , ...,   0.  ,  27.83,  27.69],
       [ 26.29,   0.  ,  24.82, ...,   0.  ,  26.49,  26.42],
       [ 25.97,   0.  ,  24.53, ...,   0.  ,  26.19,  26.16],
       ..., 
       [ 20.7 ,  20.63,  20.63, ...,  20.62,  20.63,  20.63],
       [ 19.85,  19.82,  19.83, ...,  19.82,  19.82,  19.82],
       [ 18.88,  18.87,  18.88, ...,  18.86,  18.88,  18.87]])

In [64]:
sequences = []
targets = []
row_count = 0
for dt, row in features_df.iterrows():
    pred_hour_index = features_df.index.get_loc(dt)
    if pred_hour_index - 72 >= 0:
        targets.append(features_df.iloc[pred_hour_index][:100])
        sequence = []
        sequence.append(features_df.iloc[pred_hour_index - 24])
        sequence.append(features_df.iloc[pred_hour_index - 48])
        sequence.append(features_df.iloc[pred_hour_index - 72])
        # sequence = [self.features_df.iloc[pred_hour_index - i] for i in np.arange(24, 72)]
        sequences.append(sequence)
    row_count += 1
    if row_count >= 200:
        break


In [65]:
sequences = np.array(sequences)
targets = np.array(targets)

In [78]:
train_size = 0.8
val_size = 0.1
test_size = 0.1
train_features = []
train_targets = []
test_features = []
test_targets =[]
val_features=[]
val_targets=[]
total_num_samples = sequences.shape[0]
train_boundary = int(total_num_samples*train_size)
val_boundary = int(total_num_samples*(train_size + test_size))
train_indices = np.arange(0, train_boundary)
val_indices = np.arange(train_boundary, val_boundary)
test_indices = np.arange(val_boundary, total_num_samples)
for i in train_indices:
    train_features.append(sequences[i, :, :])
    train_targets.append(targets[i])
x_train = np.array(train_features)
y_train = np.array(train_targets)
for i in test_indices:
    test_features.append(sequences[i, :, :])
    test_targets.append(targets[i])
x_test = np.array(test_features)
y_test = np.array(test_targets)
for i in val_indices:
    val_features.append(sequences[i, :, :])
    val_targets.append(targets[i])
x_val = np.array(val_features)
y_val = np.array(val_targets)
y_train.shape[1]

100

In [79]:
model = Sequential()
model.add(LSTM(50, input_dim = x_train.shape[2], input_length = x_train.shape[1]))
model.add(Dense(y_train.shape[1], init = 'zero', activation = 'linear'))
model.compile(loss='mean_squared_error', optimizer='RMSprop')
model.fit(x_train, y_train, validation_data=(x_val, y_val))

Train on 102 samples, validate on 13 samples
Epoch 1/10
102/102 [==============================] - 0s - loss: 0.0925 - val_loss: 0.0651
Epoch 2/10
102/102 [==============================] - 0s - loss: 0.0443 - val_loss: 0.0226
Epoch 3/10
102/102 [==============================] - 0s - loss: 0.0151 - val_loss: 0.0126
Epoch 4/10
102/102 [==============================] - 0s - loss: 0.0080 - val_loss: 0.0121
Epoch 5/10
102/102 [==============================] - 0s - loss: 0.0061 - val_loss: 0.0072
Epoch 6/10
102/102 [==============================] - 0s - loss: 0.0057 - val_loss: 0.0069
Epoch 7/10
102/102 [==============================] - 0s - loss: 0.0040 - val_loss: 0.0078
Epoch 8/10
102/102 [==============================] - 0s - loss: 0.0030 - val_loss: 0.0060
Epoch 9/10
102/102 [==============================] - 0s - loss: 0.0028 - val_loss: 0.0091
Epoch 10/10
102/102 [==============================] - 0s - loss: 0.0023 - val_loss: 0.0085


In [80]:
y_pred = model.predict(x_test)


In [81]:
y_pred = standard_scaler.inverse_transform(y_pred)
y_test = standard_scaler.inverse_transform(y_test)
y_pred_df = pd.DataFrame(y_pred, columns=df.columns)
y_test_df = pd.DataFrame(y_test, columns=df.columns)
ax = y_pred_df.plot()
ax2 = y_test_df.plot()
ax.legend_.remove()
ax2.legend_.remove()
plt.show()

In [82]:
np.sum(np.abs(y_pred - y_test))

7168.3787415654433